In [ ]:
!pip install ase==3.23.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

# install quantum espresso from conda
!conda install conda-forge::qe

✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [ ]:
# Below imports necessary functions
from IPython.display import HTML, Image

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation

from ase import Atoms
from ase.build import make_supercell, bulk, fcc111, molecule, add_adsorbate
from ase.io import read, write
from ase.visualize import view

import os
os.makedirs("output", exist_ok=True)

import numpy as np


# Below are functions necessary for viewing the structures
def view_x3d(atoms, idx=0):
    if isinstance(atoms[0], Atoms):
        # Assume this is a trajectory or struct list
        if (len(atoms) <= idx):
                print(f"The specified index exceeds the length of the trajectory. The length of the trajectory is {len(atoms)}.")
        return view(atoms[idx], viewer="x3d")
    else:
        return view(atoms, viewer="x3d")


def view_ase_atoms(atoms, rotation="0x,0y,0z", figsize=(4, 4), title="", scale=100):
    fig, ax = plt.subplots(figsize=figsize)
    write("output/tmp.png", atoms, rotation=rotation, scale=scale)
    img = mpimg.imread('output/tmp.png')
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')
    plt.show()
    os.remove('output/tmp.png')
    return


def traj_to_apng(traj, rotation='30x,30y,30z'):
    imgs = []
    for atom in traj:
        supercell = make_supercell(atom, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
        write('output/tmp.png', supercell, rotation=rotation, show_unit_cell=2)
        img = mpimg.imread('output/tmp.png')
        imgs.append(img)
    os.remove('output/tmp.png')

    fig, ax = plt.subplots()

    def update(frame):
        img = imgs[frame]
        ax.clear()
        ax.imshow(img)
        return []

    ani = FuncAnimation(fig, update, frames=len(imgs), blit=True)
    plt.close()
    return HTML(ani.to_jshtml())

In [ ]:

import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ase.io import write

def view_ase_atoms2(atoms, rotation="0x,0y,0z", figsize=(4, 4), title="",scale=100):

    """
    Visualizes ASE Atoms object and saves image to Google Drive.

    Parameters:
        atoms (ase.Atoms): The ASE atoms object to visualize.
        rotation (str): Rotation setting for rendering.
        figsize (tuple): Size of the matplotlib figure.
        title (str): Title to display above the image.
        scale (int): Atom size scaling in the rendered image.
        save_path (str): Full path to save the image (PNG format).
    """
    path="/content/drive/Shareddrives/CSC python works/Generated images/" #####******************************************************  Indicates the path to the folder where I wan tmy output saved
    file_type = ".png"
    name = title + file_type   #adds the title to the save path.
    save_path = os.path.join(path, name)


    # Create temp output folder if needed
    os.makedirs('output', exist_ok=True)

    tmp_path = "output/tmp.png"
    write(tmp_path, atoms, rotation=rotation, scale=scale)

    # Read and display the image
    img = mpimg.imread(tmp_path)
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')
    plt.show()

    # Save to Google Drive
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    write(save_path, atoms, rotation=rotation, scale=scale)
    print(f"Image saved to: {save_path}")

    # Clean up
    os.remove(tmp_path)


### Below is a function I created for running quantum expresso.  I'm using my  **3rd version** because it saves the output straight to my google drive.

In [ ]:
from ase.calculators.espresso import Espresso, EspressoProfile
from ase.optimize import BFGS
import shutil
import os

def run_qe_calculation(structure, calculation_type, pseudopotentials, calculation_label="Not_given"):
    """
    Runs a Quantum ESPRESSO calculation using ASE.

    Parameters:
        calculation_type (str): e.g., 'relax', 'scf', 'vc-relax' etc.
        pseudopotentials (dict): Mapping of elements to UPF files.
        structure (ase.Atoms): ASE Atoms object to be relaxed or calculated.

    Returns:
        float: Potential energy of the relaxed/final structure.
    """

    # Mount Google Drive ONLY if it's not already mounted
    from google.colab import drive
    try:
        drive.mount('/content/drive')
    except ValueError:
        pass  # Ignore if already mounted

    # Set the folder path on Google Drive (where I want my results to be saved)
    drive_folder = '/content/drive/Shareddrives/CSC python works/Quantum_Espresso_Results'  # Ruan and adjust the folder path as needed
    calculation_folder = os.path.join(drive_folder, calculation_label)

    # Create the folder if it does not exist
    os.makedirs(calculation_folder, exist_ok=True)

    # Set up Quantum ESPRESSO input
    input_data = {
        'control': {
            'calculation': calculation_type,
            'verbosity': 'high',
            'pseudo_dir': os.path.abspath('./'),  # Use os.path.abspath to ensure absolute path
            'restart_mode': 'from_scratch',
            'tstress': True,
            'tprnfor': True,
            'outdir': calculation_folder  # Save QE native outputs here
        },
        'system': {
            'ecutwfc': 30,
            'ecutrho': 240,
            'occupations': 'smearing',
            'smearing': 'mp',
            'degauss': 0.001,
            'nspin': 1
        },
        'electrons': {
            'diagonalization': 'david',
            'mixing_mode': 'plain',
            'mixing_beta': 0.7
        }
    }

    profile = EspressoProfile(command='/usr/local/bin/pw.x', pseudo_dir='./')  # or os.path.abspath('./') if pseudo_dir is relative

    calc = Espresso(
        profile=profile,
        input_data=input_data,
        pseudopotentials=pseudopotentials,
        kpts=(4, 4, 4),               # ************* Note from Prof. Cecil: if slab dimensions is (4,4,1) use k-point of "2,2,1", or use a gamma k-point i.e "1,1,1". We can use 4,4,4 for unit cells.
        directory=calculation_folder  # Save the output in the specified directory
    )

    structure.calc = calc

    # Run calculation
    energy = structure.get_potential_energy()

    #*************** Below code is an attempt to put all outputs into a specific folder *******########
    # Save energy summary to a text file
    output_file = os.path.join(calculation_folder, f'{calculation_label}_output.txt')
    with open(output_file, 'w') as f:
        f.write(f'Energy of the relaxed structure: {energy} eV\n')

    # Save final structure as CIF
    cif_path = os.path.join(calculation_folder, f'{calculation_label}_relaxed_structure.cif')
    write(cif_path, structure)  # Save relaxed structure

    # Redundantly ensure all generated files are in the Google Drive folder
    try:
        # Print source and destination to debug the issue
        print(f"Source directory: {calc.directory}")
        print(f"Destination directory: {calculation_folder}")

        # Copy files manually, avoid using copytree in case of special characters to avoid errors
        for item in os.listdir(calc.directory):
            s = os.path.join(calc.directory, item)
            d = os.path.join(calculation_folder, item)
            if os.path.isdir(s):
                shutil.copytree(s, d, dirs_exist_ok=True)  # If it's a directory, copy it recursively
            else:
                shutil.copy2(s, d)  # If it's a file, copy it

    except Exception as e:
        print(f"Error during file copy: {e}")

    return energy


### Below code generates a pseudopotential dictionary from all the available pseusopotential files available in the directory

In [ ]:
def generate_pseudopotential_dict(pseudo_dir='./'):
    """
    Automatically generates a pseudopotential dictionary by scanning a directory for .UPF files.

    Args:
        pseudo_dir (str): Path to the directory containing UPF files.

    Returns:
        dict: Dictionary in the form {element: filename}
    """
    pp_dict = {}
    for file in os.listdir(pseudo_dir):
        if file.endswith(".UPF"):
            element = file.split('.')[0]  # assumes element is the first part of the filename
            pp_dict[element] = file
    return pp_dict

### I run the calculation from below codes

In [ ]:
#step 1: Upload the necessary Pseudopotential files
from google.colab import files
uploaded = files.upload()

Saving Fe.pbesol-spn-kjpaw_psl.1.0.0.UPF to Fe.pbesol-spn-kjpaw_psl.1.0.0 (2).UPF


### use Below code to automatically upload all Pseudopotential files from google drive into the directory

In [ ]:
#Step 1b: Run below code to upload all the pseudopotential files from the specified folder in google drive into the collaborate
gdrive_pseudo_dir = '/content/drive/Shareddrives/CSC python works/Usable_Pseudo_files'  #*************************************************************** Update this path
colab_pseudo_dir = '/content/pseudopotentials'

# Step 3: Create the local destination folder if it doesn't exist
os.makedirs(colab_pseudo_dir, exist_ok=True)

# Step 4: Copy all files from Google Drive folder to local Colab folder
for filename in os.listdir(gdrive_pseudo_dir):
    src_file = os.path.join(gdrive_pseudo_dir, filename)
    dst_file = os.path.join(colab_pseudo_dir, filename)
    if os.path.isfile(src_file):
        shutil.copy(src_file, dst_file)

Ag.pbe-n-kjpaw_psl.1.0.0.UPF	   Pd.pbe-n-kjpaw_psl.1.0.0.UPF
Cd.pbe-n-kjpaw_psl.1.0.0.UPF	   Rh.pbesol-spn-kjpaw_psl.1.0.0.UPF
condacolab_install.log		   Ru.pbe-spn-kjpaw_psl.1.0.0.UPF
drive				   sample_data
Mo.pbesol-spn-kjpaw_psl.1.0.0.UPF  Tc.pbe-spn-kjpaw_psl.0.3.0.UPF
Nb.pbe-spn-kjpaw_psl.0.3.0.UPF	   Y.pbe-spn-kjpaw_psl.1.0.0.UPF
output				   Zr.pbesol-spn-kjpaw_psl.1.0.0.UPF


In [ ]:
#Step 2: Generate a pseudo dictionary with my function
%cd /content/pseudopotentials
pseudo_dict = generate_pseudopotential_dict(pseudo_dir='./')
print(pseudo_dict)
%cd /content

In [ ]:
#Step 2: Generate a pseudo dictionary with my function
pseudo_dict = generate_pseudopotential_dict(pseudo_dir='./')
print(pseudo_dict)

{'Fe': 'Fe.pbesol-spn-kjpaw_psl.1.0.0.UPF'}


In [ ]:
element_list = []
Original_lattice = []
Final_lattice = []
Energy = []
calc_duration = []

In [ ]:
import time
import pandas as pd


First_row = ["Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn"]
Second_row = ["Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd"]

for element in Second_row:
    Element = element
    print(element)

    # if element in df.values:
    #     print(element, "was previously run")
    #     element_list.append(element)

    # else:

    try:
      # Build unit cell
      unit_cell = bulk(Element)

      # Get original lattice parameter
      from ase.geometry import cell_to_cellpar
      cell = unit_cell.get_cell()
      a, b, c, alpha, beta, gamma = cell_to_cellpar(cell)
      Original_lattice.append(a)

      # Below save the initial structure image of the top and side view of the bulk structures to the drive
      title = Element + " top_view"
      view_ase_atoms2(slab, title=title, rotation="0x,0y,0z")
      title = Element +" side_view"
      view_ase_atoms2(slab, title=title, rotation="90x,60y,0z", figsize=(8, 8))

      # Run QE calculation and keep time
      start = time.time()
      Calc_type = "vc-relax"
      calc_label = Element + "_" + Calc_type
      E = run_qe_calculation(unit_cell, Calc_type, pseudo_dict, calc_label)
      Energy.append(E)
      end = time.time()

      #Below code gets the duration of the calculation
      duration = end - start
      hours = duration // 3600
      minutes = (duration % 3600) // 60
      seconds = duration # Assign the total duration to seconds
      remaining_seconds = seconds % 60
      print("Relaxation finished in", hours ,  'hours', minutes, "minutes, and",remaining_seconds, "seconds" )
      to_STR = str(hours) + ":" + str(minutes) + ":" + str(remaining_seconds)  #converting the time the calc took into string, so it can be appended
      calc_duration.append(to_STR)                      #************append duration of the calculation

      # Get final lattice parameter
      cell = unit_cell.get_cell()
      a, b, c, alpha, beta, gamma = cell_to_cellpar(cell)
      Final_lattice.append(a)

      element_list.append(element)


      # Below overwrites the initial structure with the converged image of the top and side view of the bulk structures to the drive
      title = Element + " top_view"
      view_ase_atoms2(slab, title=title, rotation="0x,0y,0z")

      title = Element +" side_view"
      view_ase_atoms2(slab, title=title, rotation="90x,60y,0z", figsize=(8, 8))


    except Exception as e:
      print(f"{element} failed with error: {e}")
      element_list.append("failed")
      Original_lattice.append("failed")
      Energy.append("failed")
      Final_lattice.append("failed")

  # # Below code uploads the results of the completed calcualtion into the sheet in real time.
  # import pandas as pd

  #   df = pd.DataFrame({
  #       'First_Row': element_list,
  #       'Original_lattice': Original_lattice,
  #       'Final_lattice': Final_lattice,
  #       'Energy': Energy,
  #       'Calcualtion Duration': calc_duration
  #   })

  # excel_path = '/content/drive/Shareddrives/CSC python works/Quantum_Espresso_Results/Lattice.xlsx'
  # sheet_name = 'Second Row'  # Change this to your desired sheet name

  # with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
  #     df.to_excel(writer, sheet_name=sheet_name, index=False)

Y
Y was previously run
Zr
Zr was previously run
Nb
Nb was previously run
Mo
Mo was previously run
Tc
Tc was previously run
Ru
Ru was previously run
Rh
Rh was previously run
Pd
Pd was previously run
Ag
Ag was previously run
Cd
Cd was previously run


In [ ]:
# element_list_2 = []
# Original_lattice_2 = []
# Final_lattice_2 = []
# Energy_2 = []

# for i in np.arange(0,10):
#     element_list_2.append(element_list[i])
#     Original_lattice_2.append(Original_lattice[i])
#     Final_lattice_2.append(Final_lattice[i])
#     Energy_2.append(Energy[i])

# element_list = element_list_2
# Original_lattice = Original_lattice_2
# Final_lattice = Final_lattice_2
# Energy = Energy_2

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'First_Row': element_list,
    'Original_lattice': Original_lattice,
    'Final_lattice': Final_lattice,
    'Energy': Energy
})

excel_path = '/content/drive/Shareddrives/CSC python works/Quantum_Espresso_Results/Lattice.xlsx'
sheet_name = 'Second Row'  # Change this to your desired sheet name

with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
print(df)

  First_Row  Original_lattice  Final_lattice        Energy
0         Y          3.650000       3.650000  -7689.816829
1        Zr          3.230000       3.230000  -8247.070044
2        Nb          2.857884       2.857884  -4547.823626
3        Mo          2.727980       2.727980  -4891.482380
4        Tc          2.740000       2.740000 -10783.901215
5        Ru          2.700000       2.700000 -11744.136375
6        Rh          2.687006       2.687006  -6330.680621
7        Pd          2.750645       2.750645  -4504.562423
8        Ag          2.892067       2.892067  -4832.350805
9        Cd          2.980000       2.980000 -10388.731050


###Below code runs individual elements that failed to run in the above loop

In [ ]:
#Step 3: create the unit cell structrue

Element = "Fe"
# unit_cell = bulk(Element)
unit_cell = bulk(Element, "bcc", a=2.866) # get lattice parameter from ChatGPT


#Below calculates the original lattice parameter of the created unit cell from this:
from ase.geometry import cell_to_cellpar`
cell = unit_cell.get_cell()
a, b, c, alpha, beta, gamma = cell_to_cellpar(cell)
print('The original lattice parameter of ' + Element + ' unit cell is:', a, "Å")

The original lattice parameter of Fe unit cell is: 2.482028807246201 Å


# Why are BCC unit cells not working

In [ ]:
print(unit_cell.get_cell())

Cell([[-1.435, 1.435, 1.435], [1.435, -1.435, 1.435], [1.435, 1.435, -1.435]])


That means ASE is generating the unit cell in a non-orthogonal form, which is expected behavior for a BCC crystal.

The bulk("Fe", "bcc", a=2.87) command does use 2.87 Å as the cubic lattice constant — but ASE internally constructs a primitive cell, not a conventional cubic cell. Therefore the unit_cell.get_cell() returns the primitive cell vectors, not the conventional cubic lattice vectors.



In the BCC lattice, the primitive vectors are:

a1 = (-a/2,  a/2,  a/2)
a2 = ( a/2, -a/2,  a/2)
a3 = ( a/2,  a/2, -a/2)

You're seeing BCC primitive cell, not cubic.

To get back the cubic lattice parameter: a_cubic = a_primitive * 2/sqrt(3)



# Use following code for BCC cells to get correct 'a' lattice paramater

In [ ]:
import numpy as np

primitive_vector = a
a_cubic = 2 / np.sqrt(3) * a
print(f"Correct cubic lattice parameter a = {a_cubic:.4f} Å")

# Step 4

In [ ]:
title = Element + " top_view"
view_ase_atoms2(slab, title=title, rotation="0x,0y,0z")

title = Element +" side_view"
view_ase_atoms2(slab, title=title, rotation="90x,60y,0z", figsize=(8, 8))

NameError: name 'slab' is not defined

In [ ]:
#Step 4: carry out a vc-relax on the unit cell
import time
start = time.time()
Calc_type = "vc-relax"
calc_label = Element + "_" + Calc_type
E = run_qe_calculation(unit_cell,Calc_type,pseudo_dict, calc_label)
end = time.time()


#Below code gets the duration of the calculation
duration = end - start
hours = duration // 3600
minutes = (duration % 3600) // 60
seconds = duration # Assign the total duration to seconds
remaining_seconds = seconds % 60
print("Relaxation finished in", hours ,  'hours', minutes, "minutes, and",remaining_seconds, "seconds" )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Source directory: /content/drive/Shareddrives/CSC python works/Quantum_Espresso_Results/Fe_vc-relax
Destination directory: /content/drive/Shareddrives/CSC python works/Quantum_Espresso_Results/Fe_vc-relax
Error during file copy: ['<', 'D', 'i', 'r', 'E', 'n', 't', 'r', 'y', ' ', "'", 'w', 'f', 'c', 'u', 'p', '1', '.', 'h', 'd', 'f', '5', "'", '>', ' ', 'a', 'n', 'd', ' ', "'", '/', 'c', 'o', 'n', 't', 'e', 'n', 't', '/', 'd', 'r', 'i', 'v', 'e', '/', 'S', 'h', 'a', 'r', 'e', 'd', 'd', 'r', 'i', 'v', 'e', 's', '/', 'C', 'S', 'C', ' ', 'p', 'y', 't', 'h', 'o', 'n', ' ', 'w', 'o', 'r', 'k', 's', '/', 'Q', 'u', 'a', 'n', 't', 'u', 'm', '_', 'E', 's', 'p', 'r', 'e', 's', 's', 'o', '_', 'R', 'e', 's', 'u', 'l', 't', 's', '/', 'F', 'e', '_', 'v', 'c', '-', 'r', 'e', 'l', 'a', 'x', '/', 'p', 'w', 's', 'c', 'f', '.', 's', 'a', 'v', 'e', '/', 'w', 'f', 'c', 'u', 'p', '

In [ ]:
# Calcualte the lattice parameter of the vc-relaxed unit cell:
from ase.geometry import cell_to_cellpar
# Access the cell attribute of the unit_cell object
cell = unit_cell.get_cell()
a, b, c, alpha, beta, gamma = cell_to_cellpar(cell)
print('The relaxed lattice parameter of ' + Element + ' unit cell is:', a, "Å")

The relaxed lattice parameter of Fe unit cell is: 2.485492908861339 Å


In [ ]:
print(E)

-4444.08040174402
